In [2]:
from pydantic import BaseModel, Field
 
# Supervisor agent tools
class ToTCSAssistant(BaseModel):
    """Transfers work to a specialized assistant to handle TCS related queries."""
 
    request: str = Field(
        description="Any necessary followup questions the TCS assistant should clarify before proceeding."
    )
 
class ToWEBAssistant(BaseModel):
    """Transfers work to a specialized assistant to handle WEB and non TCS related queries."""
 
    request: str = Field(
        description="Any necessary followup questions the WEB assistant should clarify before proceeding."
    )
 
supervisor_agent_tools = [ToTCSAssistant, ToWEBAssistant]
supervisor_tool_names = ["ToTCSAssistant", "ToWEBAssistant"]
 
"""
Load RAG database
"""
 
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceBgeEmbeddings
 
# Initialize embedding model
model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
 
tcsdb = Chroma(persist_directory="concepts", embedding_function=embeddings)
webdb = Chroma(persist_directory="webdocdb", embedding_function=embeddings)



C:\Users\local_ZSACIIH\Temp\20\ipykernel_40772\214472991.py:32: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(
c:\Users\ZSACIIH\genaispace\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\local_ZSACIIH\Temp\20\ipykernel_40772\214472991.py:38: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma packag

In [3]:

"""
Module containing a custom function to create a ChatOpenAI object.
"""
from langchain_openai import ChatOpenAI
from openai import OpenAI
import httpx


def get_eli_chat_model(temperature: float = 0.0, model_name: str = "qwen2.5-7b") -> ChatOpenAI:
    """
    Create a ChatOpenAI instance, with the specified temperature and using the specified model.

    Args:
        temperature (float, optional): The temperature value to use. Defaults to 0.0.
        model_name (str, optional): The name of the model to use. Defaults to "qwen2.5-7b".

    Returns:
        ChatOpenAI: An instance of the ChatOpenAI class.

    Raises:
        OpenAPIError: If there is an error with the API request.
    """
def get_eli_chat_model(temperature: float = 0.0, model_name: str = "qwen2.5-7b"):
    # Create an instance of the OpenAI client
    client = OpenAI(
        api_key="eli-fe3e07a3-8b92-493b-82e4-4500b80a562c",
        base_url="https://gateway.eli.gaia.gic.ericsson.se/api/openai/v1",
        http_client=httpx.Client(verify=False),
    )
    # Create an instance of ChatOpenAI
    llm = ChatOpenAI(
        model=model_name,
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
        api_key="eli-fe3e07a3-8b92-493b-82e4-4500b80a562c",
        base_url="https://gateway.eli.gaia.gic.ericsson.se/api/openai/v1",
    )
    # Now we plug the OpenAI client into our langchain-openai interface
    llm.client = client.chat.completions
    return llm

llm = get_eli_chat_model()
llm.invoke("What is LLM powered automooous agent?")

AIMessage(content='LLM-powered autonomous agents are AI-driven entities that use large language models to understand, generate, and respond to natural language inputs. These agents can operate autonomously, making decisions and performing tasks without constant human intervention. They leverage the capabilities of LLMs for complex understanding and interaction in various applications like customer service, virtual assistants, or content generation.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 108, 'prompt_tokens': 9, 'total_tokens': 117, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Qwen2.5-7b', 'system_fingerprint': 'eli', 'id': 'eli-chatcmpl-1752049943', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--5c1e4f7e-c078-4f31-b6fa-69eb87951955-0', usage_metadata={'input_tokens': 9, 'output_tokens': 108, 'total_tokens': 117, 'input_token_details': {}, 'output_token_details':

In [4]:
from langgraph.graph import MessagesState
from typing import Annotated
from langgraph.graph.message import AnyMessage, add_messages
 
class AgentState(MessagesState):
    messages: Annotated[list[AnyMessage], add_messages]

In [5]:
class Prompt:
    def __init__(self, name: str, prompt: str) -> None:
        self.name = name
        self.__prompt = prompt
 
    @property
    def prompt(self) -> str:
        return self.__prompt
 
    def __str__(self) -> str:
        return self.prompt
 
    def __repr__(self) -> str:
        return self.__str__()
   
# --- JIRA Agent ---
__SUPERVISOR_AGENT = """
You are a supervisor assistant that delegates the tasks to the TCS and WEB agents based on the given input.
"""
 
SUPERVISOR_AGENT_PROMPT = Prompt(
    name="supervisor_agent_prompt",
    prompt=__SUPERVISOR_AGENT,
)

__TCS_AGENT = """ You are a helpful TCS assistant that is specialized in answering queries based on TCS RAG database.
Based on the user query ansewer the user query with the correct answer. Use provided tools to answer the user query.
"""
TCS_AGENT_PROMPT = Prompt(
    name="tcs_agent_prompt",
    prompt=__TCS_AGENT,
)
 
 
__WEB_AGENT = """ You are a helpful assistant that is specialized in answering queries based on Web data RAG database.
Based on the user query answer the user query with the correct answer. Use provided tools to fetch and answer the user query.
"""
WEB_AGENT_PROMPT = Prompt(
    name="web_agent_prompt",
    prompt=__WEB_AGENT,
)

In [6]:
from langchain.tools import tool

@tool
def search_tcs_rag(query: str) -> str:
    """Searches the Knowledgebase for the given query and returns the result."""
    results = tcsdb.similarity_search_with_score(query, k=3)  # Search top 3 docs (adjust as needed)
   
    for doc, score in results:
        result_doc = doc.page_content.split(":")[0].strip()
    return result_doc
tcs_agent_tools = [search_tcs_rag]
tcs_agent_tool_names = ["search_tcs_rag"]

In [7]:
from langchain.tools import tool

@tool
def search_web_rag(query: str) -> str:
    """Searches the Knowledgebase for the given query and returns the result."""
    results = webdb.similarity_search_with_score(query, k=3)  # Search top 3 docs (adjust as needed)
   
    for doc, score in results:
        result_doc = doc.page_content.split(":")[0].strip()
    return result_doc
web_agent_tools = [search_web_rag]
web_agent_tool_names = ["search_web_rag"]

In [8]:
from langchain.prompts.chat import ChatPromptTemplate, MessagesPlaceholder
 
def get_supervisor_agent_chain():
    model = get_eli_chat_model()
    model = model.bind_tools(supervisor_agent_tools)
    system_message = SUPERVISOR_AGENT_PROMPT
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_message.prompt),
            MessagesPlaceholder(variable_name="messages"),
        ],
        template_format="jinja2",
    )
 
    return prompt | model
 
# --- TCS Agent ---
def get_tcs_agent_chain():
    model = get_eli_chat_model()
    model = model.bind_tools(tcs_agent_tools)
    system_message = TCS_AGENT_PROMPT
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_message.prompt),
            MessagesPlaceholder(variable_name="messages"),
        ],
        template_format="jinja2",
    )
 
    return prompt | model
 
# --- WEB Agent ---
def get_web_agent_chain():
    model = get_eli_chat_model()
    model = model.bind_tools(web_agent_tools)
    system_message = WEB_AGENT_PROMPT
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_message.prompt),
            MessagesPlaceholder(variable_name="messages"),
        ],
        template_format="jinja2",
    )
 
    return prompt | model

In [9]:
from langgraph.prebuilt import tools_condition
from langgraph.graph import END
 
def route_supervisor_agent(state: AgentState,):
    route = tools_condition(state)
    print("route_supervisor_agent:", route)
    if route == END:
        return END
    tool_calls = state["messages"][-1].tool_calls
    if tool_calls:
        if tool_calls[0]["name"] == "ToTCSAssistant":
            return "enter_tcs_agent"
        elif tool_calls[0]["name"] == "ToWEBAssistant":
            return "enter_web_agent"
        return "supervisor_agent_tools"
    raise ValueError("Invalid route")
 
# --- TCS Agent ---
def route_tcs_agent(state: AgentState,):
    route = tools_condition(state)
    print("route_tcs_agent:", route)
    if route == END:
        return END
    tool_calls = state["messages"][-1].tool_calls
    if tool_calls:
        if tool_calls[0]["name"] in tcs_agent_tool_names:
            return "tcs_agent_tools"
    raise ValueError("Invalid route")
 
# --- WEB Agent ---
def route_web_agent(state: AgentState,):
    route = tools_condition(state)
    print("route_web_agent:", route)
    if route == END:
        return END
    tool_calls = state["messages"][-1].tool_calls
    if tool_calls:
        if tool_calls[0]["name"] in web_agent_tool_names:
            return "web_agent_tools"
    raise ValueError("Invalid route")

In [10]:
from langgraph.prebuilt import ToolNode
from langchain_core.runnables import RunnableLambda
from langchain_core.messages import ToolMessage
 
def handle_tool_error(state) -> dict:
    error = state.get("error")
    tool_calls = state["messages"][-1].tool_calls
    return {
        "messages": [
        ToolMessage(
            content=f"Error: {repr(error)}\n please fix your mistakes.",
            tool_call_id=tc["id"],
            )
            for tc in tool_calls
        ]
    }
 
#---Supervisor tools node---
def supervisor_tool_node_with_fallback(_) -> dict:
        return ToolNode(supervisor_agent_tools).with_fallbacks(
            [RunnableLambda(handle_tool_error)], exception_key="error"
    )
 
#---TCS tools node---
def tcs_tool_node_with_fallback(_) -> dict:
        return ToolNode(tcs_agent_tools).with_fallbacks(
            [RunnableLambda(handle_tool_error)], exception_key="error"
    )
 
#---WEB tools node---
def web_tool_node_with_fallback(_) -> dict:
        return ToolNode(web_agent_tools).with_fallbacks(
            [RunnableLambda(handle_tool_error)], exception_key="error"
    )
 
 
#---Supervisor node---
def supervisor_node(state: AgentState):
    supervisor_chain = get_supervisor_agent_chain()
 
    while True:
        response = supervisor_chain.invoke(state)
        print("jira_node:", response)
        if not response.tool_calls and (
            not response.content
            or isinstance(response.content, list)
            and not response.content[0].get("text")
        ):
            messages = state["messages"] + [("user", "Respond with a real output.")]
            state = {**state, "messages": messages}
        else:
            break
 
    return {"messages": response}
 
#---TCS node---
def tcs_node(state: AgentState):
    tcs_chain = get_tcs_agent_chain()
 
    while True:
        response = tcs_chain.invoke(state)
        print("tcs_node:", response)
        if not response.tool_calls and (
            not response.content
            or isinstance(response.content, list)
            and not response.content[0].get("text")
        ):
            messages = state["messages"] + [("user", "Respond with a real output.")]
            state = {**state, "messages": messages}
        else:
            break
 
    return {"messages": response}
 
#---WEB node---
def web_node(state: AgentState):
    web_chain = get_web_agent_chain()
 
    while True:
        response = web_chain.invoke(state)
        print("web_node:", response)
        if not response.tool_calls and (
            not response.content
            or isinstance(response.content, list)
            and not response.content[0].get("text")
        ):
            messages = state["messages"] + [("user", "Respond with a real output.")]
            state = {**state, "messages": messages}
        else:
            break
 
    return {"messages": response}